<a href="https://colab.research.google.com/github/duskvirkus/colab-notebooks/blob/main/StyleGAN2_activations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## StyleGAN2 Activations

Notebook that generates activations of a StyleGAN2 rosinality model.

Convert pkl model: https://github.com/dvschultz/stylegan2-ada-pytorch/blob/main/SG2_ADA_PT_to_Rosinality.ipynb

Thanks to [Derrick Schultz](https://github.com/dvschultz) for the notebook this is based on. Can be found at: [https://github.com/dvschultz/stylegan2-ada-pytorch/blob/eps/Advanced_StyleGAN_Network_bending.ipynb](https://github.com/dvschultz/stylegan2-ada-pytorch/blob/eps/Advanced_StyleGAN_Network_bending.ipynb)

## Prep

In [1]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-a46fdea8-1bad-ac1e-17ea-63ace40870d9)


In [2]:
# Install libraries
!git clone -b audio-animate https://github.com/dvschultz/network-bending
!pip uninstall torch torchvision -y
!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install Ninja kmeans-pytorch
!apt-get install -y vim make gdb libopencv-dev
!wget https://download.pytorch.org/libtorch/cu101/libtorch-shared-with-deps-1.5.0%2Bcu101.zip
!unzip /content/libtorch-shared-with-deps-1.5.0+cu101.zip -d /root/
%cd network-bending

#build custom pytorch transformations
!chmod +x /content/network-bending/build_custom_transforms.sh
!/content/network-bending/build_custom_transforms.sh /root/libtorch/

Cloning into 'network-bending'...
remote: Enumerating objects: 369, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 369 (delta 22), reused 21 (delta 9), pack-reused 332
Receiving objects: 100% (369/369), 21.44 MiB | 32.91 MiB/s, done.
Resolving deltas: 100% (213/213), done.
Found existing installation: torch 1.9.0+cu102
Uninstalling torch-1.9.0+cu102:
  Successfully uninstalled torch-1.9.0+cu102
Found existing installation: torchvision 0.10.0+cu102
Uninstalling torchvision-0.10.0+cu102:
  Successfully uninstalled torchvision-0.10.0+cu102
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 703.8 MB 22 kB/s 
     |████████████████████████████████| 6.6 MB 63.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires tor

In [5]:
!gdown --id 1rL-J63eFfn80IYU2GfVY977GI2qOG6dw -O /content/ladiesblack.pt # Model Credit: Derrick Schultz

Downloading...
From: https://drive.google.com/uc?id=1rL-J63eFfn80IYU2GfVY977GI2qOG6dw
To: /content/ladiesblack.pt
133MB [00:01, 105MB/s] 


## Fix existing script

In [6]:
%%writefile generate_activations.py

import argparse
import torch
import yaml
import os
import copy

from torchvision import utils
from model import Generator
from tqdm import tqdm
from util import *

def generate(args, g_ema, device, mean_latent, t_dict_list):
    with torch.no_grad():
        g_ema.eval()
        for i in tqdm(range(args.pics)):
            extra_t_dict_list =  copy.deepcopy(t_dict_list)
            extra_t_dict_list.append({'layerID': -1, 'index': i})
            sample_z = torch.randn(args.sample, args.latent, device=device)
            sample, _ = g_ema([sample_z], 
                                truncation=args.truncation, 
                                truncation_latent=mean_latent, 
                                transform_dict_list=extra_t_dict_list)
            if not os.path.exists('sample'):
                    os.makedirs('sample')
            utils.save_image(
                sample,
                f'sample/{str(i).zfill(6)}.png',
                nrow=1,
                normalize=True,
                range=(-1, 1))


if __name__ == '__main__':
    device = 'cuda'

    parser = argparse.ArgumentParser()

    parser.add_argument('--size', type=int, default=1024)
    parser.add_argument('--sample', type=int, default=1)
    parser.add_argument('--pics', type=int, default=20)
    parser.add_argument('--truncation', type=float, default=0.5)
    parser.add_argument('--truncation_mean', type=int, default=4096)
    parser.add_argument('--ckpt', type=str, default="stylegan2-ffhq-config-f.pt")
    parser.add_argument('--channel_multiplier', type=int, default=2)
    parser.add_argument('--config', type=str, default="configs/empty_transform_config.yaml")

    args = parser.parse_args()

    args.latent = 512
    args.n_mlp = 8

    yaml_config = {}
    with open(args.config, 'r') as stream:
        try:
            yaml_config = yaml.load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    
    g_ema = Generator(
        args.size, args.latent, args.n_mlp, channel_multiplier=args.channel_multiplier
    ).to(device)
    new_state_dict = g_ema.state_dict()
    checkpoint = torch.load(args.ckpt)
    
    ext_state_dict  = torch.load(args.ckpt)['g_ema']
    g_ema.load_state_dict(checkpoint['g_ema'])
    new_state_dict.update(ext_state_dict)
    g_ema.load_state_dict(new_state_dict)
    g_ema.eval()
    g_ema.to(device)

    if args.truncation < 1:
        with torch.no_grad():
            mean_latent = g_ema.mean_latent(args.truncation_mean)
    else:
        mean_latent = None
    
    layer_channel_dims = create_layer_channel_dim_dict(args.channel_multiplier, 16)
    transform_dict_list = create_transforms_dict_list(yaml_config, None, layer_channel_dims)
    generate(args, g_ema, device, mean_latent, transform_dict_list)
    
    config_out = {}
    config_out['transforms'] = yaml_config['transforms']
    with open(r'sample/config.yaml', 'w') as file:
        documents = yaml.dump(config_out, file)

Overwriting generate_activations.py


## Run the script

Activations can be found `/content/network-bending/activations` or download the activations.zip made by the last cell.

In [7]:
!python generate_activations.py --size 1024 --ckpt /content/ladiesblack.pt --pics 1 --config /content/network-bending/configs/example_transform_config.yaml --truncation 0.8 --channel_multiplier 2

  0% 0/1 [00:00<?, ?it/s]Scalar multiply parameter should be a float
100% 1/1 [00:40<00:00, 40.29s/it]


In [9]:
!zip -r activations.zip activations

Streaming output truncated to the last 5000 lines.
  adding: activations/7/0/469.png (deflated 2%)
  adding: activations/7/0/082.png (deflated 3%)
  adding: activations/7/0/179.png (stored 0%)
  adding: activations/7/0/451.png (deflated 2%)
  adding: activations/7/0/505.png (deflated 0%)
  adding: activations/7/0/426.png (deflated 1%)
  adding: activations/7/0/148.png (deflated 2%)
  adding: activations/7/0/254.png (deflated 2%)
  adding: activations/7/0/173.png (stored 0%)
  adding: activations/7/0/472.png (deflated 2%)
  adding: activations/7/0/431.png (deflated 2%)
  adding: activations/7/0/373.png (deflated 1%)
  adding: activations/7/0/425.png (stored 0%)
  adding: activations/7/0/509.png (deflated 1%)
  adding: activations/7/0/172.png (deflated 2%)
  adding: activations/7/0/273.png (stored 0%)
  adding: activations/7/0/470.png (deflated 2%)
  adding: activations/7/0/486.png (deflated 0%)
  adding: activations/7/0/090.png (deflated 1%)
  adding: activations/7/0/125.png (stored 0%)